In [1]:
import configparser
import numpy as np
import os
import sys
import tensorflow as tf

In [2]:
# set to latest model version number

def set_model_version_number():
    version_number = []
    global MODEL_VERSION
    global MODEL_PATH

    if os.path.exists(os.path.join(MODEL_SAVE_DIRECTORY,MODEL_NAME)):   
        for entry in os.listdir(os.path.join(MODEL_SAVE_DIRECTORY,MODEL_NAME)):
            version_number.append(entry)       
        MODEL_VERSION = version_number[-1]
        MODEL_PATH = os.path.join(MODEL_SAVE_DIRECTORY, MODEL_NAME, MODEL_VERSION)
        

In [3]:

config = configparser.ConfigParser()
config.read('config/main.conf')

DATASET = 1
MODEL_VERSION =  "0001"
DOWNLOAD_GOOGLE_LM = False

if DATASET == 1:
    set_dataset = "imdb"
if DATASET == 2:
    set_dataset = "s140"

DATASET_URL = (config[set_dataset]['DATASET_URL'])

DATASET_FOLDER = config[set_dataset]['DATASET_FOLDER']
DATASET_TAR_FILE_NAME = config[set_dataset]['DATASET_TAR_FILE_NAME']
DATASET_NAME = config[set_dataset]['DATASET_NAME']

MODEL_NAME = config[set_dataset]['MODEL_NAME']

CLEAN_DATA_FILE = os.path.join(DATASET_FOLDER,"normalized_dataset.csv")
TAR_FILE_PATH = os.path.join(DATASET_FOLDER,DATASET_TAR_FILE_NAME)
DATA_SET_LOCATION = os.path.join(DATASET_FOLDER,DATASET_NAME)

MODEL_SAVE_DIRECTORY = config[set_dataset]['MODEL_SAVE_DIRECTORY']
# Create the model save directory
if not os.path.exists(MODEL_SAVE_DIRECTORY):
    os.makedirs(MODEL_SAVE_DIRECTORY)
    
IMAGE_SAVE_FOLDER = config[set_dataset]['IMAGE_SAVE_FOLDER']
    
GLOVE_EMBEDDINGS = config[set_dataset]['GLOVE_EMBEDDINGS']
COUNTER_FITTED_VECTORS = config[set_dataset]['COUNTER_FITTED_VECTORS']

GLOVE_EMBEDDINGS_MATRIX = config[set_dataset]['GLOVE_EMBEDDINGS_MATRIX']
COUNTER_FITTED_EMBEDDINGS_MATRIX = config[set_dataset]['COUNTER_FITTED_EMBEDDINGS_MATRIX']

LM_URLS = config[set_dataset]['LM_URLS']
LM_DIRECTORY = config[set_dataset]['LM_DIRECTORY']

####### files required to reconstruct the final trained model ##############################
MODEL_PATH = os.path.join(MODEL_SAVE_DIRECTORY, MODEL_NAME, MODEL_VERSION)

set_model_version_number()

ASSESTS_FOLDER = os.path.join(MODEL_PATH,"assets")
MODEL_ASSETS_VOCABULARY_FILE = os.path.join(ASSESTS_FOLDER,"vocab")
MODEL_ASSETS_EMBEDDINGS_FILE = os.path.join(ASSESTS_FOLDER,"imdb_glove_embeddings_matrix")
MODEL_ASSETS_COUNTER_EMBEDDINGS_FILE = os.path.join(ASSESTS_FOLDER,"counter_embeddings_matrix")
MODEL_ASSETS_DISTANCE_MATRIX = os.path.join(ASSESTS_FOLDER,"distance_matrix.npy")
MODEL_ASSETS_SAVE_BEST_WEIGHTS = os.path.join(ASSESTS_FOLDER, "cp.ckpt")
MODEL_TRAINING_HISORTY_FILE = os.path.join(ASSESTS_FOLDER, "training_history.csv")



### load our pre trained sentiment model

In [4]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization # in Tensorflow 2.1 and above
import pickle 

MAX_VOCABULARY_SIZE = 50000
DIMENSION = 300
LEARNING_RATE = 1e-4


from manny_modules import tf_normalize_data as tfnd
from manny_modules import return_model as rmodel

saved_vocab = pickle.load(open(MODEL_ASSETS_VOCABULARY_FILE, 'rb'))
saved_word_index = dict(zip(saved_vocab, range(len(saved_vocab))))

saved_embeddings_matric = pickle.load(open(MODEL_ASSETS_EMBEDDINGS_FILE, 'rb'))


vectorizer_layer = TextVectorization(
    standardize=tfnd.normlize_data, 
    max_tokens=MAX_VOCABULARY_SIZE, 
    output_mode='int',
    output_sequence_length=300)

# build vocabulary, will also run the normalize_data() 
vectorizer_layer.set_vocabulary(saved_vocab)


saved_model = rmodel.create_model(vectorizer_layer,
                                  saved_embeddings_matric,
                                  saved_vocab,
                                  dimension=DIMENSION, 
                                  lrate=LEARNING_RATE)

# load the weights
saved_model.load_weights(MODEL_ASSETS_SAVE_BEST_WEIGHTS) # loads best weights saved during training

### check predictions for unseen data

In [5]:
# check negative review
p_2 = [["Seriously, don't bother if you're over 12. This looks like a kids show designed purely to sell merchandise, theme park rides, etc. No logic, holes all over the shop, no characters motivation and really crap acting to top it off... just rubbish, really"]]
prob_positive = saved_model.predict(p_2)

print("Positive confidence: ",prob_positive, " Negative confidence: ", (1 - prob_positive ))


# check positive review
p = [["It's one thing to bring back elements, characters, settings and stories, and to flash them in front of the audience to cash in on the nostalgia and/or recognisable memorabilia but without using it to further the plot and other to do exactly the opposite. It was about time that Star Wars directives understood that it is too unique a product to be lend to corporate filmmakers. Star Wars needs to be understood and its uniqueness has to be acknowledged in order to make the new stories feel like they belong. This may sound too obvious but if you ever wondered why the new SW movies are so controversial this may be the reason.Like with 'Spider-Man: Into the Spider-verse (2018)' and their comicbook-industry experts participation, the creators behind The Mandalorian were experts of the industry, connoisseurs of the Star Wars Universe and even long time fans. So they were able to not only recapture the aesthetic of the grimy, battered Star Wars but also build upon it taking the most 'subtle' things into account. Things like the predominancy of puppets and practical effects over CGI, settings you can feel and touch over green screens and the abundancy of not only known elements previously seen in Star Wars, but a whole batch of new creatures, designs and overall plot elements that felt like they belong to this universe and had always been there. Exceeding expectations are not only the visual aspects but the narrative too. It might be too late for some story elements now, but it is of great importance that from now on you try to watch the unraveling of the story unspoiled. I was lucky to have seen the premiere of the show before the 'memefication' of a certain 'element' that went viral and became one of the biggest highlights of the show. But for me I saw the reveal of this element unspoiled and I was pleasantly shocked, a memory I'll always carry with me. The ability of these creators to generate such shock value and deep moments it's often baffling to me. This is proof that the creators behind the narrative are fully aware of the complexities of the universe they are tampering with and like an experienced surgeon, they are able to tweak, traverse and call back any Star Wars element as they please and with astonishing results."]]
prob_positive = saved_model.predict(p)

print("Positive confidence: ",prob_positive, " Negative confidence: ", (1 - prob_positive ))

Positive confidence:  [[0.01855881]]  Negative confidence:  [[0.9814412]]
Positive confidence:  [[0.9553545]]  Negative confidence:  [[0.04464549]]


### load the distance matrix from disk (load this before running below tests)
- This is a large file (~20GB), so will take time to load

In [ ]:

import numpy as np

distance_matrix = np.load(MODEL_ASSETS_DISTANCE_MATRIX)



# Download pre-trained Google Language Model

### load the urls from a file and store in a list

In [ ]:

google_urls = []

with open(LM_URLS, 'r') as f:
    for line in f:
        google_urls.append(line.rstrip()) # remove new line character


### download the files and store in the ```LM_DIRECTORY``` directory 

In [29]:
from manny_modules import download_utils as dutils

if DOWNLOAD_GOOGLE_LM:
    for u in google_urls:
        dutils.download(u, dest_folder=LM_DIRECTORY)

### load pre-trained google language model

In [1]:
from manny_modules.google_lm import data_utils 
from google.protobuf import text_format
from manny_modules.google_lm.language_model import LanguageModel

In [2]:
glm = LanguageModel()

Loaded language model vocabulary!


Recovering graph.


INFO:tensorflow:Recovering Graph language_model/graph-2016-09-10.pbtxt


Recovering checkpoint language_model/ckpt-*


### use pre-trained google language model

In [12]:
from manny_modules import nearest_neighbour as nn

target_word = saved_word_index['movie']

nearest_neighbour, distance_to_neighbour = nn.closest_neighbours(target_word, distance_matrix, number_of_words_to_return=4, max_distance=None)

closest_word = [saved_vocab[x] for x in nearest_neighbour]

print("Words closest to `%s` are `%s` " % (saved_vocab[target_word], closest_word))

In [13]:
# stop returning 0 if word is not in the vocabulary


prefix = 'this'
suffix = 'was'

language_model_predictions = glm.get_words_probs(prefix, closest_word, suffix)
#print("Most likely word is: ", closest_word[np.argmax(language_model_predictions)])
